# Preparation

In [1]:
import json
import requests
from pathlib import Path
import pandas as pd
import time
import asyncio
import aiohttp
import glob
from bs4 import BeautifulSoup
import re
from typing import Any, Optional, Union, Dict, Coroutine, Tuple

# Get East Java Province

## Get raw data

In [ ]:
response = requests.get('https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=050000&semester_id=20251')
data =json.loads(response.text)

In [ ]:
df_jatim_province = pd.DataFrame(data)
df_jatim_province

In [ ]:
for idx, row in df_jatim_province.iterrows():
    # get all `kecamatan` for each `kabupaten/kota`
    kode_kab = row['kode_wilayah']
    nama_kab = row['nama']
    url_kab = f'https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=2&kode_wilayah={kode_kab.strip()}&semester_id=20251'
    response_kab = requests.get(url_kab)
    data_kab = json.loads(response_kab.text)
    df_kab = pd.DataFrame(data_kab)

    Path(f'../data/dapo-kemdikdasmen/Jawa-Timur/{nama_kab}').mkdir(parents=True, exist_ok=True)

    for idx2, row2 in df_kab.iterrows():
        # get all `sekolah` for each `kecamatan`
        kode_kec = row2['kode_wilayah']
        nama_kec = row2['nama']

        url_kec = f'https://dapo.kemendikdasmen.go.id/rekap/progresSP-slb?id_level_wilayah=3&kode_wilayah={kode_kec.strip()}&semester_id=20251'
        response_kec = requests.get(url_kec)
        data_kec = json.loads(response_kec.text)
        df_kec = pd.DataFrame(data_kec)

        path_kec = f'../data/dapo-kemdikdasmen/Jawa-Timur/{nama_kab}/{nama_kec}'
        Path(path_kec).mkdir(parents=True, exist_ok=True)
        
        with open(f'{path_kec}/ALL.json', 'w') as f:
            json.dump(data_kec, f, indent=4)

        for idx3, row3, in df_kec.iterrows():
            # get detail information for each `sekolah`
            nama_sekolah = row3['nama']
            kode_sekolah_enkrip = row3['sekolah_id_enkrip']

            url_sekolah = f'https://dapo.kemendikdasmen.go.id/rekap/sekolahDetail?semester_id=20251&sekolah_id={kode_sekolah_enkrip.strip()}'

            response_sekolah = requests.get(url_sekolah)
            data_sekolah = json.loads(response_sekolah.text)

            with open(f'../data/dapo-kemdikdasmen/Jawa-Timur/{nama_kab}/{nama_kec}/{nama_sekolah}.json', 'w') as f:
                json.dump(data_sekolah, f, indent=4)

## Add acreditation data for each schools

In [ ]:
async def fetch_akreditasi_sekolah(
    session: aiohttp.ClientSession,
    sekolah: pd.Series
) -> None:
    """
    Fetch school accreditation asynchronously
    """
    url = 'https://referensi.data.kemendikdasmen.go.id/snpmb/site/sekolah'
    req_param = { 'npsn': sekolah['npsn'] }
    
    try:
        timeout = aiohttp.ClientTimeout(total=10)
        async with session.get(url, params=req_param, timeout=timeout) as response:
            html = await response.text()
    
        # extract data with BeautifulSoup library
        soup = BeautifulSoup(html, 'lxml')
        
        td_akreditasi_title = soup.find('td', string='Akreditasi')
        
        if td_akreditasi_title is None:
            print(f"[{sekolah['nama']}] Accreditation title not found!")
            return
        
        td_akreditasi_value = td_akreditasi_title.find_next('td')
        
        if td_akreditasi_value is None:
            print(f"[{sekolah['nama']}] Accreditation value not found!")
            return
        
        tag_akreditasi_value = td_akreditasi_value.find('a')
        
        if tag_akreditasi_value is None:
            print(f"[{sekolah['nama']}] Accreditation tag value not found!")
            return
        
        akreditasi = tag_akreditasi_value.text.strip()
        
        sekolah_json_path = f'../data/dapo-kemdikdasmen/Prov. Jawa Timur/{sekolah['induk_kabupaten']}/{sekolah['induk_kecamatan']}/{sekolah['nama']}.json'
        
        with open(sekolah_json_path, 'r') as f:
            sekolah_json = json.load(f)
        
        # get first item in sekolah json
        sekolah_record = sekolah_json[0]
        
        # delete existing key `akreditasi` in `sekolah_json`
        sekolah_record.pop("akreditasi", None)
        
        # store new sekolah json file in single data in list/array
        new_sekolah_record = {}
        for key, val in sekolah_record.items():
            new_sekolah_record[key] = val
            
            # place `akreditasi` column after specific column
            if key == 'rombel':
                new_sekolah_record['akreditasi'] = akreditasi
        
        # replace the old dict in the list
        sekolah_json[0] = new_sekolah_record
        
        with open(sekolah_json_path, "w") as f:
            json.dump(sekolah_json, f, indent=4)
        
        print(f"[{sekolah['nama']}-{sekolah['induk_kecamatan']}-{sekolah['induk_kabupaten']}] accreditation: {akreditasi}")
    except Exception as e:
        print(f"[{sekolah['nama']}-{sekolah['induk_kecamatan']}-{sekolah['induk_kabupaten']}] Failed: {e}")

async def get_all_jatim_schools_accreditation() -> None:
    files_jatim_sekolah_each_kecamatan = glob.glob('../data/dapo-kemdikdasmen/Prov. Jawa Timur/*/*/ALL.json')

    tasks = []
    
    async with aiohttp.ClientSession() as session:
        for sekolah_kec in files_jatim_sekolah_each_kecamatan:
            df_sekolah_kec = pd.read_json(sekolah_kec)
            for _, sekolah in df_sekolah_kec.iterrows(): 
                tasks.append(fetch_akreditasi_sekolah(session=session, sekolah=sekolah))
        
        await asyncio.gather(*tasks, return_exceptions=True)

await get_all_jatim_schools_accreditation()

[SDLB ABCD PGRI 3 CLURING-Kec. Cluring-Kab. Banyuwangi] accreditation: A
[SMALB ABCD PGRI 2 JAJAG-Kec. Gambiran-Kab. Banyuwangi] accreditation: B
[SLB BINA INSANI-Kec. Wongsorejo-Kab. Banyuwangi] accreditation: B
[SLB BUDI MULIA -Kec. Garum-Kab. Blitar] accreditation: B
[SLB PELITA HATI-Kec. Kesamben-Kab. Jombang] accreditation: B
[SLB-C.D NGUDI HAYU-Kec. Srengat-Kab. Blitar] accreditation: A
[SLB PERMATA HATI-Kec. Megaluh-Kab. Jombang] accreditation: C
[SLB TUNAS HARAPAN III-Kec. Mojoagung-Kab. Jombang] accreditation: B
[SMALB PGRI GENTENG-Kec. Genteng-Kab. Banyuwangi] accreditation: B
[SLB ABCD KURNIA ASIH-Kec. Ngoro-Kab. Jombang] accreditation: B
[SLB HARDIKA BAKTI-Kec. Peterongan-Kab. Jombang] accreditation: B
[SLB TUNAS HARAPAN II PETERONGAN-Kec. Peterongan-Kab. Jombang] accreditation: B
[SLB PELITA BANGSA-Kec. Kesamben-Kab. Jombang] accreditation: B
[SLB TUNAS HARAPAN I TEMBELANG-Kec. Tembelang-Kab. Jombang] accreditation: B
[SLB NEGERI BALONGSARI JOMBANG-Kec. Megaluh-Kab. Jomban

## Add `sekolah_id`, `npsn`, `bentuk_pendidikan`, and `status_sekolah` to all East Java schools

In [48]:
files_jatim_sekolah_each_kecamatan = glob.glob('../data/dapo-kemdikdasmen/Prov. Jawa Timur/*/*/ALL.json')

for kec_path in files_jatim_sekolah_each_kecamatan:
    with open(kec_path, 'r') as f:
        kec_schools_json = json.load(f)
    
    kec_dir = Path(kec_path).parent
    
    school_list = [
        f for f in glob.glob(f'{str(kec_dir)}/*.json')
        if Path(f).name != 'ALL.json'
    ]
    
    for school_path in school_list:
        school_source_json = next((s for s in kec_schools_json if s['nama'] == Path(school_path).stem), None)
        
        
        if school_source_json is None:
            kota = Path(school_path).parent.parent.name
            kecamatan = Path(school_path).parent.name
            sekolah = Path(school_path).stem
            print(f'School source for "{sekolah}"-{kecamatan}-{kota} is not found!')
            continue
        
        with open(school_path, 'r') as f:
            sekolah_json = json.load(f)
        
        
        # get first item in sekolah json
        sekolah_record = sekolah_json[0]
        
        # store new sekolah json file in single data in list/array
        new_sekolah_record = {}
        for key, val in sekolah_record.items():
            new_sekolah_record[key] = val
            
            # place `akreditasi` column after specific column
            if key == 'rombel':
                new_sekolah_record['sekolah_id'] = school_source_json['sekolah_id']
                new_sekolah_record['npsn'] = school_source_json['npsn']
                new_sekolah_record['bentuk_pendidikan'] = school_source_json['bentuk_pendidikan']
                new_sekolah_record['status_sekolah'] = school_source_json['status_sekolah']
        
        # replace the old dict in the list
        sekolah_json[0] = new_sekolah_record
        
        with open(school_path, 'w') as f:
            json.dump(sekolah_json, f, indent=4)
        

# Get all province data

## Synchronously

In [24]:
df_all_province = []

for i in range(1, 40):
    req_param = {
        'id_level_wilayah': 1,
        'kode_wilayah': f'{i:02}0000',
        'semester_id': 20251,
    }
    
    success = False
    for attempt in range(3):  # max 3 retries
        try:
            response = requests.get(
                'https://dapo.kemendikdasmen.go.id/rekap/progres-slb',
                params=req_param,
                timeout=10
            )
            response.raise_for_status()  # raise error if not 200 OK
            data = response.json()       # directly parse JSON
            success = True
            break
        except Exception as e:
            print(f"Attempt {attempt+1} failed for index {i}: {e}")
            time.sleep(2)

    if not success:
        print(f"Skipping index {i} after 3 failed attempts.\n")
        continue

    if not data:
        print(f"Index {i} returned empty response. Skipping.\n")
        continue
    
    print(f'{i}')
    print(data)
    print('\n')
    
    df_province = pd.DataFrame(data)
    
    if df_province.empty:
        print(f"Index {i} returned empty DataFrame. Skipping.\n")
        continue
    
    province_name = df_province['induk_provinsi'].iloc[0]
    
    df_all_province.append(df_province)
    
    # Path(f'../data/dapo-kemdikdasmen/{province_name}').mkdir(parents=True, exist_ok=True)

# store to file csv for persistent
for i, df in enumerate(df_all_province):
    df.to_csv(f"../data/dapo-kemdikdasmen/metadata/provinsi/df_{i}.csv", index=False)

print('Get all province data is done!!!')

1
[{'nama': 'Kota Adm. Jakarta Barat', 'kode_wilayah': '016200  ', 'id_level_wilayah': 2, 'mst_kode_wilayah': '010000  ', 'induk_provinsi': 'Prov. D.K.I. Jakarta', 'kode_wilayah_induk_provinsi': '010000  ', 'induk_kabupaten': None, 'kode_wilayah_induk_kabupaten': None, 'kirim_total': 21, 'sekolah_total': 21, 'persen': 100}, {'nama': 'Kota Adm. Jakarta Pusat', 'kode_wilayah': '016000  ', 'id_level_wilayah': 2, 'mst_kode_wilayah': '010000  ', 'induk_provinsi': 'Prov. D.K.I. Jakarta', 'kode_wilayah_induk_provinsi': '010000  ', 'induk_kabupaten': None, 'kode_wilayah_induk_kabupaten': None, 'kirim_total': 6, 'sekolah_total': 6, 'persen': 100}, {'nama': 'Kota Adm. Jakarta Timur', 'kode_wilayah': '016400  ', 'id_level_wilayah': 2, 'mst_kode_wilayah': '010000  ', 'induk_provinsi': 'Prov. D.K.I. Jakarta', 'kode_wilayah_induk_provinsi': '010000  ', 'induk_kabupaten': None, 'kode_wilayah_induk_kabupaten': None, 'kirim_total': 25, 'sekolah_total': 25, 'persen': 100}, {'nama': 'Kota Adm. Jakarta Se

## Asynchronously

In [19]:
df_all_province = []

async def fetch_with_retries(
    session: aiohttp.ClientSession,
    url: str,
    params: dict[str, Any],
    retries: int = 3,
    delay: int = 2
) -> Optional[list[dict[str, Any]]]:
    """Fetch data with retries using aiohttp"""
    for attempt in range(retries):
        try:
            timeout = aiohttp.ClientTimeout(total=10)
            async with session.get(url, params=params, timeout=timeout) as resp:
                resp.raise_for_status()

                # Try standard JSON first
                try:
                    return await resp.json()
                except Exception:
                    # Fall back: get raw text
                    text = await resp.text()

                    # Quick fix: replace single quotes with double quotes
                    # (only outside numbers/booleans/null)
                    cleaned = re.sub(r"'", '"', text)

                    try:
                        return json.loads(cleaned)
                    except json.JSONDecodeError:
                        print(f"⚠️ Could not parse response for {params['kode_wilayah']}")
                        return None

        except Exception as e:
            print(f"Attempt {attempt+1} failed for {params['kode_wilayah']}: {e}")
            await asyncio.sleep(delay)
    print(f"Skipping {params['kode_wilayah']} after {retries} failed attempts.\n")
    return None

async def fetch_all() -> None:
    url = "https://dapo.kemendikdasmen.go.id/rekap/progres-slb"
    tasks = []

    async with aiohttp.ClientSession() as session:
        for i in range(1, 40):
            req_param = {
                'id_level_wilayah': 1,
                'kode_wilayah': f'{i:02}0000',
                'semester_id': 20251,
            }
            tasks.append(fetch_with_retries(session, url, req_param))

        results = await asyncio.gather(*tasks)

        for i, data in enumerate(results, start=1):
            if not data:
                print(f"Index {i} returned empty response. Skipping.\n")
                continue

            print(f"{i}")
            print(data)
            print("\n")

            df_province = pd.DataFrame(data)

            if df_province.empty:
                print(f"Index {i} returned empty DataFrame. Skipping.\n")
                continue

            province_name = df_province['induk_provinsi'].iloc[0]
            df_all_province.append(df_province)

            # Path(f'../data/dapo-kemdikdasmen/{province_name}').mkdir(parents=True, exist_ok=True)

await fetch_all()

Attempt 1 failed for 020000: 404, message='Not Found', url='https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=020000&semester_id=20251'
Attempt 1 failed for 210000: 404, message='Not Found', url='https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=210000&semester_id=20251'
Attempt 1 failed for 290000: 404, message='Not Found', url='https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=290000&semester_id=20251'
Attempt 1 failed for 070000: 404, message='Not Found', url='https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=070000&semester_id=20251'
Attempt 1 failed for 120000: 404, message='Not Found', url='https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=120000&semester_id=20251'
Attempt 1 failed for 050000: 404, message='Not Found', url='https://dapo.kemendikdasmen.go.id/rekap/progres-slb?id_level_wilayah=1&kode_wilayah=050000

# Get All Schools Data For Each Province Synchronously

## Helper functions

In [ ]:
def safe_request(
    url: str,
    params: dict[str, Union[str, int]],
    max_retries: int = 3,
    delay: int = 2
) -> Optional[Union[list[dict[str, Any]], dict[str, Any]]]:
    """
    A helper function to safely request to target API endpoint with proper error handler
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            return data
        except Exception as e:
            print(f"[ERROR] Attempt {attempt+1}/{max_retries} failed: {e}")
            time.sleep(delay)
    return None

## Main execution

In [5]:
failed_requests = []

In [7]:
for df_province in df_all_province:
    provinsi = df_province['induk_provinsi'].iloc[0]

    for idx, row in df_province.iterrows():
        kode_kab = row['kode_wilayah']
        nama_kab = row['nama']

        # skip if kab dir already exists (resume feature)
        kab_path = Path(f'../data/dapo-kemdikdasmen/{provinsi}/{nama_kab}')
        kab_path.mkdir(parents=True, exist_ok=True)

        # request kabupaten → kecamatan
        url_kab = 'https://dapo.kemendikdasmen.go.id/rekap/progres-slb'
        req_param_kab = {
            'id_level_wilayah': 2,
            'kode_wilayah': kode_kab.strip(),
            'semester_id': 20251
        }
        data_kab = safe_request(url_kab, req_param_kab)
        if not data_kab:
            failed_requests.append((provinsi, nama_kab, "kabupaten", kode_kab))
            continue
        df_kab = pd.DataFrame(data_kab)

        for idx2, row2 in df_kab.iterrows():
            kode_kec = row2['kode_wilayah']
            nama_kec = row2['nama']

            kec_path = Path(f'../data/dapo-kemdikdasmen/{provinsi}/{nama_kab}/{nama_kec}')
            kec_path.mkdir(parents=True, exist_ok=True)

            # skip if ALL.json already exists
            all_file = kec_path / "ALL.json"
            if all_file.exists():
                print(f"[SKIP] {all_file} already exists.")
                with open(all_file, 'r') as f:
                    data_kec = json.load(f)
            else:
                url_kec = 'https://dapo.kemendikdasmen.go.id/rekap/progresSP-slb'
                req_param_kec = {
                    'id_level_wilayah': 3,
                    'kode_wilayah': kode_kec.strip(),
                    'semester_id': 20251
                }
                data_kec = safe_request(url_kec, req_param_kec)
                if not data_kec:
                    failed_requests.append((provinsi, nama_kab, nama_kec, "kecamatan", kode_kec))
                    continue

                with open(all_file, 'w', encoding='utf-8') as f:
                    json.dump(data_kec, f, indent=4, ensure_ascii=False)

            df_kec = pd.DataFrame(data_kec)

            for idx3, row3 in df_kec.iterrows():
                nama_sekolah = row3['nama'].replace("/", "-")  # sanitize filename
                kode_sekolah_enkrip = row3['sekolah_id_enkrip']

                sekolah_file = kec_path / f"{nama_sekolah}.json"
                if sekolah_file.exists():
                    print(f"[SKIP] {sekolah_file} already exists.")
                    continue

                url_sekolah = 'https://dapo.kemendikdasmen.go.id/rekap/sekolahDetail'
                req_param_sekolah = {
                    'semester_id': 20251,
                    'sekolah_id': kode_sekolah_enkrip.strip()
                }
                data_sekolah = safe_request(url_sekolah, req_param_sekolah)
                if not data_sekolah:
                    failed_requests.append((provinsi, nama_kab, nama_kec, nama_sekolah, "sekolah"))
                    continue

                with open(sekolah_file, 'w', encoding='utf-8') as f:
                    json.dump(data_sekolah, f, indent=4, ensure_ascii=False)

# --- save failures log ---
if failed_requests:
    with open("failed_requests.json", "w", encoding="utf-8") as f:
        json.dump(failed_requests, f, indent=4, ensure_ascii=False)
    print(f"\n[INFO] Some requests failed. Logged to failed_requests.json")
else:
    print("\n[INFO] All requests completed successfully!")

[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Cengkareng\ALL.json already exists.
[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Cengkareng\SLB B-C ALFIANY.json already exists.
[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Cengkareng\SLB CAHAYA BINTANG.json already exists.
[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Grogol Petamburan\ALL.json already exists.
[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Grogol Petamburan\SLB DHARMA ASIH.json already exists.
[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Kali Deres\ALL.json already exists.
[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Kali Deres\SLB NEGERI 10 .json already exists.
[SKIP] ..\data\dapo-kemdikdasmen\Prov. D.K.I. Jakarta\Kota Adm. Jakarta Barat\Kec. Kali Deres\SLB B KUN

KeyboardInterrupt: 

# Get All Schools Data For Each Province Asynchronously

## Helper functions

In [13]:
async def fetch(
    session: aiohttp.ClientSession,
    url: str,
    params: Optional[Dict[str, Any]],
    retries: int = 3,
    delay: int = 2
) -> Optional[Any]:
    """
    Perform an asynchronous HTTP GET request with retries.

    Args:
        session (aiohttp.ClientSession): The active aiohttp session.
        url (str): The request URL.
        params (Optional[Dict[str, Any]]): Query parameters for the request.
        retries (int, optional): Number of retry attempts if the request fails. Defaults to 3.
        delay (int): The delay if request error

    Returns:
        Optional[Any]: Parsed JSON response if successful, otherwise None.
    """
    for attempt in range(retries):
        try:
            timeout = aiohttp.ClientTimeout(total=10)
            async with session.get(url, params=params, timeout=timeout) as response:
                if response.status != 200:
                    print(f"[ERROR] {url} returned status {response.status}")
                    continue

                text = await response.text()
                try:
                    return json.loads(text)  # try to parse as JSON
                except json.JSONDecodeError:
                    print(f"[WARNING] Response not JSON at {url}, returning raw text")
                    return {"raw_text": text}  # fallback as string
        except Exception as e:
            print(f"[ERROR] Attempt {attempt+1} failed: {e}")
            if attempt < retries - 1:
                await asyncio.sleep(delay)
    return None

async def fetch_and_save(
    session: aiohttp.ClientSession,
    url: str,
    params: dict,
    save_path: Path,
    meta: tuple
) -> None:
    """
    Perform an asynchronous HTTP GET request, save the response to a JSON file, 
    and log failures if the request fails.

    Args:
        session (aiohttp.ClientSession): Active aiohttp session for making requests.
        url (str): The target URL for the request.
        params (dict): Query parameters for the request.
        save_path (Path): File path where the response JSON will be saved.
        meta (tuple): Metadata to record in `failed_requests` if the request fails.
                      Example: (provinsi, nama_kab, nama_kec, nama_sekolah, "sekolah")

    Returns:
        None: Writes the response to `save_path` if successful, 
              otherwise appends the metadata to `failed_requests`.
    """
    data = await fetch(session, url, params)
    if not data:
        failed_requests.append(meta)
        return

    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def insert_dash(s: str, pos: int) -> str:
    """
    Add dash in a string in specific position
    """
    return s[:pos] + '-' + s[pos:]

In [ ]:
async def fetch_kota_detail_data(
    session: aiohttp.ClientSession,
    nama_provinsi: str,
    semester_id: int,
    kode_kota: str,
    nama_kota: str,
) -> Tuple[Optional[Path], Optional[pd.DataFrame]]:
    """
    Fetch kota detail data
    """
    url_kab = 'https://dapo.kemendikdasmen.go.id/rekap/progres-slb'
    req_param_kab = {
        'id_level_wilayah': 2,
        'kode_wilayah': kode_kota.strip(),
        'semester_id': semester_id
    }
    data_kab = await fetch(session, url_kab, req_param_kab)
    if not data_kab:
        failed_requests.append((
            nama_provinsi, 
            nama_kota, 
            'kabupaten', 
            kode_kota
        ))
        return None, None

    semester_dir_str = insert_dash(str(semester_id), pos=4)
    kota_path = Path(f'../data/dapo-kemdikdasmen/{semester_dir_str}/{nama_provinsi}/{nama_kota}')
    kota_path.mkdir(parents=True, exist_ok=True)
    
    return kota_path, pd.DataFrame(data_kab)
    

async def fetch_kec_detail_data(
    session: aiohttp.ClientSession,
    nama_provinsi: str,
    semester_id: int,
    nama_kota: str,
    kota_path: Path,
    kode_kec: str,
    nama_kec: str,
) -> Tuple[Optional[Path], Optional[pd.DataFrame]]:
    """
    Fetch kecamatan detail data
    """
    kec_path = kota_path / nama_kec

    all_file = kec_path / "ALL.json"
    if all_file.exists():
        print(f"[SKIP] {all_file} already exists.")
        with open(all_file, 'r', encoding='utf-8') as f:
            data_kec = json.load(f)
    else:
        url_kec = 'https://dapo.kemendikdasmen.go.id/rekap/progresSP-slb'
        req_param_kec = {
            'id_level_wilayah': 3,
            'kode_wilayah': kode_kec.strip(),
            'semester_id': semester_id
        }
        data_kec = await fetch(session, url_kec, req_param_kec)
        if not data_kec:
            failed_requests.append((
                nama_provinsi, 
                nama_kota, 
                nama_kec, 
                'kecamatan', 
                kode_kec
            ))
            return None, None
        
        kec_path.mkdir(parents=True, exist_ok=True)
        
        with open(all_file, 'w', encoding='utf-8') as f:
            json.dump(data_kec, f, indent=4, ensure_ascii=False)

    return kec_path, pd.DataFrame(data_kec)

def fetch_sekolah_detail_data(
    session: aiohttp.ClientSession,
    nama_provinsi: str,
    nama_kota: str,
    nama_kec: str,
    semester_id: int,
    sekolah_tasks: list[Coroutine],
    nama_sekolah: str,
    kode_sekolah_enkrip: str,
    sekolah_file: Path,
) -> None:
    """
    Fetch sekolah detail data
    """
    if sekolah_file.exists():
        print(f"[SKIP] {sekolah_file} already exists.")
        return

    url_sekolah = 'https://dapo.kemendikdasmen.go.id/rekap/sekolahDetail'
    req_param_sekolah = {
        'semester_id': semester_id,
        'sekolah_id': kode_sekolah_enkrip.strip()
    }

    sekolah_tasks.append(
        fetch_and_save(
            session=session, url=url_sekolah, params=req_param_sekolah,
            save_path=sekolah_file, 
            meta=(nama_provinsi, nama_kota, nama_kec, nama_sekolah, "sekolah")
        )
    )

## Main execution

In [ ]:
# new main execution
failed_requests = []

async def get_all_indonesian_slb(
    session: aiohttp.ClientSession,
    semester_id: int
) -> None:
    async with aiohttp.ClientSession() as session:
        for df_province in df_all_province:
            nama_provinsi = df_province['induk_provinsi'].iloc[0]
            
            for _, row in df_province.iterrows():
                kode_kota = str(row['kode_wilayah'])
                nama_kota = row['nama'].strip()
                
                kota_path, df_kota = await fetch_kota_detail_data(
                    session=session, 
                    nama_provinsi=nama_provinsi,
                    semester_id=semester_id, 
                    kode_kota=kode_kota, 
                    nama_kota=nama_kota
                )
                
                if df_kota is None or kota_path is None:
                    continue
                
                for _, row2 in df_kota.iterrows():
                    kode_kec = row2['kode_wilayah']
                    nama_kec = row2['nama'].strip()
                    
                    kec_path, df_kec = await fetch_kec_detail_data(
                        session=session, 
                        nama_provinsi=nama_provinsi,
                        semester_id=semester_id,
                        nama_kota=nama_kota,
                        kota_path=kota_path,
                        kode_kec=kode_kec,
                        nama_kec=nama_kec
                    )
                    
                    if df_kec is None or kec_path is None:
                        continue

                    
                    # sekolah requests: run concurrently per kecamatan
                    sekolah_tasks = []
                    for _, row3 in df_kec.iterrows():
                        nama_sekolah = row3['nama'].replace("/", "-").strip()
                        kode_sekolah_enkrip = row3['sekolah_id_enkrip']
                        sekolah_file = kec_path / f"{nama_sekolah}.json"
                        
                        fetch_sekolah_detail_data(
                            session=session,
                            nama_provinsi=nama_provinsi,
                            nama_kota=nama_kota,
                            nama_kec=nama_kec,
                            semester_id=semester_id,
                            sekolah_tasks=sekolah_tasks,
                            nama_sekolah=nama_sekolah,
                            kode_sekolah_enkrip=kode_sekolah_enkrip,
                            sekolah_file=sekolah_file
                        )
                    
                    # execute all sekolah_tasks concurrently
                    if sekolah_tasks:
                        await asyncio.gather(*sekolah_tasks)
    
    if failed_requests:
        with open("failed_requests.json", "w", encoding="utf-8") as f:
            json.dump(failed_requests, f, indent=4, ensure_ascii=False)
        print(f"\n[INFO] Some requests failed. Logged to failed_requests.json")
    else:
        print("\n[INFO] All requests completed successfully!")

In [ ]:
# # this is old cell code
# failed_requests = []

# async def get_all_indonesian_slb(semester_id: int) -> None:
#     connector = aiohttp.TCPConnector(limit=50)
    
#     async with aiohttp.ClientSession(connector=connector) as session:
#         for df_province in df_all_province:
#             provinsi_name = df_province['induk_provinsi'].iloc[0]

#             for _, row in df_province.iterrows():
#                 kode_kab = row['kode_wilayah']
#                 nama_kab = row['nama'].strip()

#                 semester_dir_str = insert_dash(str(semester_id), pos=4)
#                 kab_path = Path(f'../data/dapo-kemdikdasmen/{semester_dir_str}/{provinsi}/{nama_kab}')
#                 kab_path.mkdir(parents=True, exist_ok=True)

#                 # kabupaten → kecamatan
#                 url_kab = 'https://dapo.kemendikdasmen.go.id/rekap/progres-slb'
#                 req_param_kab = {
#                     'id_level_wilayah': 2,
#                     'kode_wilayah': kode_kab.strip(),
#                     'semester_id': semester_id
#                 }
#                 data_kab = await fetch(session, url_kab, req_param_kab)
#                 if not data_kab:
#                     failed_requests.append((provinsi, nama_kab, "kabupaten", kode_kab))
#                     continue
#                 df_kab = pd.DataFrame(data_kab)

#                 for _, row2 in df_kab.iterrows():
#                     kode_kec = row2['kode_wilayah']
#                     nama_kec = row2['nama'].strip()

#                     kec_path = kab_path / nama_kec
#                     kec_path.mkdir(parents=True, exist_ok=True)

#                     all_file = kec_path / "ALL.json"
#                     if all_file.exists():
#                         print(f"[SKIP] {all_file} already exists.")
#                         with open(all_file, 'r', encoding='utf-8') as f:
#                             data_kec = json.load(f)
#                     else:
#                         url_kec = 'https://dapo.kemendikdasmen.go.id/rekap/progresSP-slb'
#                         req_param_kec = {
#                             'id_level_wilayah': 3,
#                             'kode_wilayah': kode_kec.strip(),
#                             'semester_id': semester_id
#                         }
#                         data_kec = await fetch(session, url_kec, req_param_kec)
#                         if not data_kec:
#                             failed_requests.append((provinsi, nama_kab, nama_kec, "kecamatan", kode_kec))
#                             continue
#                         with open(all_file, 'w', encoding='utf-8') as f:
#                             json.dump(data_kec, f, indent=4, ensure_ascii=False)

#                     df_kec = pd.DataFrame(data_kec)

#                     # sekolah requests: run concurrently per kecamatan
#                     sekolah_tasks = []
#                     for _, row3 in df_kec.iterrows():
#                         nama_sekolah = row3['nama'].replace("/", "-").strip()
#                         kode_sekolah_enkrip = row3['sekolah_id_enkrip']
#                         sekolah_file = kec_path / f"{nama_sekolah}.json"

#                         if sekolah_file.exists():
#                             print(f"[SKIP] {sekolah_file} already exists.")
#                             continue

#                         url_sekolah = 'https://dapo.kemendikdasmen.go.id/rekap/sekolahDetail'
#                         req_param_sekolah = {
#                             'semester_id': semester_id,
#                             'sekolah_id': kode_sekolah_enkrip.strip()
#                         }

#                         sekolah_tasks.append(
#                             fetch_and_save(
#                                 session=session, url=url_sekolah, params=req_param_sekolah,
#                                 save_path=sekolah_file, 
#                                 meta=(provinsi, nama_kab, nama_kec, nama_sekolah, "sekolah")
#                             )
#                         )

#                     if sekolah_tasks:
#                         await asyncio.gather(*sekolah_tasks)
        
#     if failed_requests:
#         with open("failed_requests.json", "w", encoding="utf-8") as f:
#             json.dump(failed_requests, f, indent=4, ensure_ascii=False)
#         print(f"\n[INFO] Some requests failed. Logged to failed_requests.json")
#     else:
#         print("\n[INFO] All requests completed successfully!")

In [ ]:
# get all indonesian slb only at year 2025 semester 1
async with aiohttp.ClientSession() as session:
    await get_all_indonesian_slb(session=session, semester_id=20251)

## Get data from the past 5 years

In [10]:
def generate_semester_ids(years_back: int, current_year: int, current_sem: int) -> list[int]:
    semesters = []
    year, sem = current_year, current_sem
    
    for _ in range(years_back * 2):
        semesters.append(int(f"{year}{sem}"))
        # go backward one semester
        if sem == 1:
            year -= 1
            sem = 2
        else:
            sem = 1
    return semesters


In [16]:
# semester_ids = generate_semester_ids(5, 2025, 1)
# semester_ids.append(20201)
# semester_ids = [20232, 20231]
semester_ids = [20222, 20221]
# semester_ids = [20212, 20211]
# semester_ids = [20202, 20201]

In [34]:
metadata_provinsi_path = '../data/dapo-kemdikdasmen/metadata/provinsi'
all_province_list = glob.glob(f'{metadata_provinsi_path}/*')

df_all_province = [pd.read_csv(f'{metadata_provinsi_path}/df_{i}.csv') for i in range(len(all_province_list))]

async with aiohttp.ClientSession() as session:
    for sem_id in semester_ids:
        print(f"\n=== Fetching semester {sem_id} ===")
        await get_all_indonesian_slb(session=session, semester_id=sem_id)


=== Fetching semester 20222 ===
kode kota: 
16200
<class 'str'>
[ERROR] Attempt 1 failed: 
kode kota: 
16000
<class 'str'>
[ERROR] Attempt 1 failed: 
[ERROR] Attempt 2 failed: 
[ERROR] Attempt 3 failed: 
kode kota: 
16400
<class 'str'>
[ERROR] Attempt 1 failed: 
kode kota: 
16300
<class 'str'>
[ERROR] Attempt 1 failed: 
kode kota: 
16100
<class 'str'>
kode kota: 
22200
<class 'str'>
kode kota: 
20500
<class 'str'>
kode kota: 
21400
<class 'str'>
[ERROR] Attempt 1 failed: 
kode kota: 
20700
<class 'str'>
kode kota: 
21700
<class 'str'>
[ERROR] Attempt 1 failed: 
kode kota: 
21100
<class 'str'>
kode kota: 
21800
<class 'str'>
[ERROR] Attempt 1 failed: 
[ERROR] Attempt 2 failed: 
[ERROR] Attempt 3 failed: 
kode kota: 
22100
<class 'str'>
[ERROR] Attempt 1 failed: 
[ERROR] Attempt 2 failed: 
[ERROR] Attempt 3 failed: 
kode kota: 
21500
<class 'str'>
[ERROR] Attempt 1 failed: 
kode kota: 
21600
<class 'str'>
kode kota: 
22500
<class 'str'>
kode kota: 
22000
<class 'str'>
[ERROR] Attempt 1 

CancelledError: 